In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from functions import main_cleaning

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [ ]:
df_cleaned = main_cleaning(df_raw)

In [4]:
df_cleaned = df_cleaned.dropna()         # Solo lo hago en el df_cleaned de ML

In [ ]:
df_cleaned.corr()

Results sin normnalizar -> (17.717622715597013, 12.581512831873468, 0.507506217561434)

In [43]:
features = df_cleaned[["Run", "py1", "pz1" "py2", "pz2", "is_same_charge", "is_outlier"]]
target = df_cleaned[["M"]]

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

In [45]:
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

X_train_norm= normalizer.fit_transform(X_train)        

X_test_norm = normalizer.transform(X_test)

In [46]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)   
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

# Define models
models = {

  'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),

}   

# Train and evaluate models
results = {}

for name, model in models.items():
  # Train the model
    model.fit(X_train_norm, y_train)
  
  # Make predictions
    y_pred_scaled = model.predict(X_test_norm)

    mae_standardized = mean_absolute_error(y_test, y_pred_scaled)
    mse_standardized = root_mean_squared_error(y_test, y_pred_scaled)
    r2_standardized = r2_score(y_test, y_pred_scaled)
  
  # Store results
    results[name] = {
        'mae' : mae_standardized,
        'mse' : mse_standardized,
        'R2' : r2_standardized
  }

# Print results
for name, result in results.items():
  print(f"\n{name}:")
  print(f"Mean Absolute Error: {result['mae']}")
  print(f"Root Mean Squared Error: {result['mse']}")
  print(f"R2: {result['R2']}")

# Compare R2_value
r2_values = {name: result['R2'] for name, result in results.items()}
best_model = max(r2_values, key=r2_values.get)

print("\nModel Accuracy Comparison:")
for name, accuracy in r2_values.items():
  print(f"{name}: {accuracy:.4f}")

print(f"\nBest performing model: {best_model} with accuracy {r2_values[best_model]:.4f}")

c:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Random Forest:
Mean Absolute Error: 2.3478761082736876
Root Mean Squared Error: 3.918134937140667
R2: 0.9759148902055805

Model Accuracy Comparison:
Random Forest: 0.9759

Best performing model: Random Forest with accuracy 0.9759


In [48]:
rf_pred = RandomForestRegressor(random_state=42, n_estimators=100)

In [49]:
rf_pred.fit(X_train_norm, y_train)

c:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=42)

In [50]:
y_pred_2 = rf_pred.predict(X_test_norm)

r2_standardized_2 = r2_score(y_test, y_pred_2)

In [51]:
r2_standardized_2

0.9759148902055805

In [52]:
features_norm = normalizer.transform(features)

In [53]:
target_pred = rf_pred.predict(features_norm)

c:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [54]:
r2_standardized_feat = r2_score(target, target_pred)

r2_standardized_feat

0.9923652860944597

In [55]:
df_cleaned["predicted"] = target_pred
df_cleaned

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M,is_same_charge,is_outlier,predicted
0,147115,366639895,58.71410,-7.311320,10.531000,-57.29740,12.82020,-2.202670,2.177660,1,11.28360,-1.032340,-1.88066,-11.077800,2.14537,-2.344030,-2.072810,-1,8.94841,False,False,8.994683
1,147115,366704169,6.61188,-4.152130,-0.579855,-5.11278,4.19242,-1.028420,-3.002840,-1,17.14920,-11.713500,5.04474,11.464700,12.75360,0.808077,2.734920,1,15.89300,False,False,17.865334
2,147115,367112316,25.54190,-11.480900,2.041680,22.72460,11.66100,1.420480,2.965600,1,15.82030,-1.472800,2.25895,-15.588800,2.69667,-2.455080,2.148570,1,38.38770,True,False,39.171290
3,147115,366952149,65.39590,7.512140,11.887100,63.86620,14.06190,2.218380,1.007210,1,25.12730,4.087860,2.59641,24.656300,4.84272,2.330210,0.565865,-1,3.72862,False,False,4.278652
4,147115,366523212,61.45040,2.952840,-14.622700,-59.61210,14.91790,-2.093750,-1.371540,-1,13.88710,-0.277757,-2.42560,-13.670800,2.44145,-2.423700,-1.684810,-1,2.74718,True,False,3.633771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,146511,522575834,12.31310,-10.658000,5.164440,3.36858,11.84330,0.280727,2.690370,-1,1.80181,0.668609,-1.58437,0.537805,1.71967,0.307851,-1.171470,1,8.44779,False,False,8.396937
99996,146511,522786431,18.46420,7.854990,15.133000,-7.08659,17.05020,-0.404510,1.092010,1,14.69110,-1.418020,-2.28117,-14.443500,2.68598,-2.383880,-2.126960,1,20.71540,True,False,20.307962
99997,146511,522906124,4.18566,-3.273500,-0.308507,-2.59013,3.28801,-0.723075,-3.047630,1,72.81740,-11.074900,-9.28179,-71.369300,14.45010,-2.300410,-2.444050,-1,12.71350,False,False,13.945377
99998,146511,523243830,54.46220,11.352600,11.880900,51.92400,16.43280,1.867800,0.808132,-1,8.58671,0.378009,3.07828,8.007050,3.10141,1.677170,1.448610,1,4.69670,False,False,5.282712
